# load info.mat 

In [3]:
import sys
from pathlib import Path
import numpy as np  # numpy import 추가
import pandas as pd

# ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ 사용자 제공 코드 시작 ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
ROOT = Path("/home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON")
sys.path.append(str(ROOT))

from config.utils import load_mat_info

INFO_PATH = ROOT / "data" / "KAIST" / "250325" / "cal" / "info.mat"
REF_INFO_PATH = Path("/home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON/data/DGIST/eye1_Info.mat")

info = load_mat_info(str(INFO_PATH))
ref_info = load_mat_info(str(REF_INFO_PATH))

# convert info.mat dtype into new data type minge with python code

In [20]:
# /scripts/align_info_dtypes.py
import sys
from pathlib import Path
import numpy as np
import pandas as pd

ROOT = Path("/home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON")
sys.path.append(str(ROOT))

from config.utils import load_mat_info

INFO_PATH = ROOT / "data" / "KAIST" / "250325" / "cal" / "info.mat"
REF_INFO_PATH = Path("/home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON/data/DGIST/eye1_Info.mat")

info = load_mat_info(str(INFO_PATH))
ref_info = load_mat_info(str(REF_INFO_PATH))


def _is_structured_complex(arr) -> bool:
    """(real, imag) 필드를 가진 numpy structured array인지 확인."""
    try:
        dt = getattr(arr, "dtype", None)
        if dt is None or dt.names is None:
            return False
        names = set(dt.names)
        return {"real", "imag"}.issubset(names)
    except Exception:
        return False


def _structured_to_complex(arr, target_dtype=np.complex128):
    """
    구조체 배열( real, imag ) -> 복소수 ndarray로 변환.
    target_dtype은 보통 ref의 dtype(np.complex64/128 등)을 넣어줌.
    """
    real = np.asarray(arr["real"])
    imag = np.asarray(arr["imag"])
    out = real + 1j * imag
    return out.astype(target_dtype, copy=False)


def _bytes_to_str(x):
    """bytes → str 안전 변환 (배열/스칼라 모두 처리)."""
    if isinstance(x, (bytes, bytearray)):
        try:
            return x.decode("utf-8", errors="ignore")
        except Exception:
            return str(x)

    if isinstance(x, np.ndarray) and x.dtype.type is np.bytes_:
        # 배열인 경우 벡터화 decode
        return np.vectorize(lambda b: b.decode("utf-8", errors="ignore"))(x)

    return x


def _to_ndarray_like(x):
    """리스트/스칼라도 ndarray로 일단 감싸 통일 처리 (object는 유지 가능)."""
    if isinstance(x, np.ndarray):
        return x
    # dict는 그대로 반환 (여기서 강제 변환하지 않음)
    if isinstance(x, dict):
        return x
    try:
        return np.array(x)
    except Exception:
        return x  # 변환 불가 시 원본 유지

def _finalize_shape(val, ref_val):
    """
    ref_val이 스칼라(int/float/bool/complex 등)인데
    val이 크기 1짜리 ndarray이면 스칼라로 풀어줌.
    """
    if isinstance(ref_val, (int, float, complex, bool, np.number)):
        if isinstance(val, np.ndarray) and val.size == 1:
            return val.item()  # numpy 배열 → Python 스칼라
    return val        


def _coerce_value_to_ref_dtype(val, ref_val, key):
    """
    ref_val의 dtype/타입에 맞춰 val을 변환.
    - (real, imag) structured → complex 변환 처리
    - bytes → str 처리
    - 그 외에는 numpy astype으로 캐스팅 시도
    실패 시 원본 유지하고 경고 메시지 반환
    """
    warn = None

    # 우선 bytes → str 정리
    val = _bytes_to_str(val)
    ref_val = _bytes_to_str(ref_val)

    # dict (MATLAB struct처럼 계층적) 는 여기서 무리하게 캐스팅하지 않음
    if isinstance(val, dict) or isinstance(ref_val, dict):
        return val, warn  # 스킵 (필요 시 재귀 변환 로직을 별도 작성)

    # ndarray-like 로 정규화
    v_arr = _to_ndarray_like(val)
    r_arr = _to_ndarray_like(ref_val)

    # ref가 스칼라/문자열일 수도 있음
    # 문자열 기준: ref가 문자열이면 val도 문자열로 맞춤
    if isinstance(r_arr, np.ndarray) and r_arr.dtype.type is np.str_:
        try:
            v_arr = _bytes_to_str(v_arr)
            if isinstance(v_arr, np.ndarray) and v_arr.dtype.type is not np.str_:
                v_arr = v_arr.astype(str)
            return v_arr, warn
        except Exception as e:
            warn = f"[WARN] key={key}: string 변환 실패 ({e}), 기존 값 유지"
            return val, warn

    # ref가 numpy 배열인 경우 dtype 얻기
    ref_dtype = None
    if isinstance(r_arr, np.ndarray):
        ref_dtype = r_arr.dtype

    # 1) structured (real, imag) → complex 변환
    if isinstance(v_arr, np.ndarray) and _is_structured_complex(v_arr):
        target_dtype = ref_dtype if (ref_dtype and np.issubdtype(ref_dtype, np.complexfloating)) else np.complex128
        try:
            coerced = _structured_to_complex(v_arr, target_dtype=target_dtype)
            return coerced, warn
        except Exception as e:
            warn = f"[WARN] key={key}: structured→complex 변환 실패 ({e}), 기존 값 유지"
            return val, warn

    # 2) 일반 astype 캐스팅 (ref dtype이 있는 경우)
    if ref_dtype is not None:
        try:
            # object 배열이면 요소별 캐스팅을 시도
            if isinstance(v_arr, np.ndarray) and v_arr.dtype == object:
                def _cast_item(x):
                    try:
                        # bytes → str 재시도
                        x = _bytes_to_str(x)
                        return np.array(x).astype(ref_dtype)
                    except Exception:
                        return x  # 실패한 요소는 그대로 둠

                if v_arr.size == 0:
                    return v_arr.astype(ref_dtype), warn
                vect_cast = np.vectorize(_cast_item, otypes=[ref_dtype])
                coerced = vect_cast(v_arr)
                return coerced, warn

            # 일반 케이스
            coerced = np.asarray(v_arr).astype(ref_dtype, copy=False)
            return coerced, warn
        except Exception as e:
            warn = f"[WARN] key={key}: dtype={getattr(v_arr,'dtype',type(v_arr))} -> {ref_dtype} 캐스팅 실패 ({e}), 기존 값 유지"
            return val, warn

    # ref dtype이 없는(스칼라 Python 타입 등) 케이스: 가능한 한 동일 타입으로 변환 시도
    # 예: ref가 float, int, complex, bool 등
    try:
        if isinstance(ref_val, (float, int, complex, bool, np.number)):
            coerced = np.array(v_arr).astype(type(ref_val))
            return coerced, warn
    except Exception as e:
        warn = f"[WARN] key={key}: 스칼라 기준 캐스팅 실패 ({e}), 기존 값 유지"
        return val, warn

    # 마지막: 변환 패스
    return val, warn


def align_info_dtypes(info: dict, ref_info: dict):
    """
    info의 키를 순회하며, ref_info에 동일 키가 있으면 ref_info의 dtype/타입에 맞게 변환.
    반환: (updated_info, report_df)
    """
    changes = []
    for key, val in list(info.items()):
        if key not in ref_info:
            continue

        ref_val = ref_info[key]
        before_dtype = None
        after_dtype = None

        v_arr = val if isinstance(val, np.ndarray) else _to_ndarray_like(val)
        if isinstance(v_arr, np.ndarray):
            before_dtype = str(v_arr.dtype)
        else:
            before_dtype = str(type(val))

        new_val, warn = _coerce_value_to_ref_dtype(val, ref_val, key)
        new_val = _finalize_shape(new_val, ref_val)


        v_arr2 = new_val if isinstance(new_val, np.ndarray) else _to_ndarray_like(new_val)
        if isinstance(v_arr2, np.ndarray):
            after_dtype = str(v_arr2.dtype)
        else:
            after_dtype = str(type(new_val))

        info[key] = new_val  # 갱신

        changes.append(
            {
                "key": key,
                "before": before_dtype,
                "ref": (str(getattr(_to_ndarray_like(ref_val), "dtype", type(ref_val)))),
                "after": after_dtype,
                "warn": warn if warn else "",
            }
        )

        if warn:
            print(warn)

    report = pd.DataFrame(changes, columns=["key", "before", "ref", "after", "warn"])
    return info, report


# 실행
info_aligned, report_df = align_info_dtypes(info, ref_info)

# 요약 출력
print("\n=== DTYPE ALIGN REPORT (top 30) ===")
print(report_df.head(30).to_string(index=False))

# 필요하면 CSV로 저장
# report_df.to_csv("dtype_align_report.csv", index=False)


=== DTYPE ALIGN REPORT (top 30) ===
           key                             before        ref      after warn
        FDFlip                              uint8      int64      int64     
    adaptiveBG                              uint8      int64      int64     
 adaptiveBGOsc                              uint8      int64      int64     
          bgBW                            float64      int64      int64     
   bgLineShift                            float64      int64      int64     
       bgLines                            float64      int64      int64     
        bgMean                            float64    float64    float64     
         bgOsc                            float64    float64    float64     
         bgRef                            float64    float64    float64     
 depthPerPixel                            float64    float64    float64     
      dispComp [('real', '<f8'), ('imag', '<f8')] complex128 complex128     
    kbandwidth                         

In [26]:
import torch
from pathlib import Path

SAVE_PATH_PT = Path("/home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON/data/KAIST/info_aligned.pt")

# info_aligned 객체를 .pt로 저장
torch.save(info_aligned, SAVE_PATH_PT)

print(f"✅ PT 저장 완료: {SAVE_PATH_PT}")

✅ PT 저장 완료: /home/work/OCT_DL/CDAC_OCT/CDAC_PYTHON/data/KAIST/info_aligned.pt
